<a href="https://colab.research.google.com/github/SanzarRehman/Semantic-Segmentation-CSE499-Azk-Section-6/blob/master/Train_YoloV3_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Connect google drive**

In [ ]:
# Check if NVIDIA GPU is enabled
!nvidia-smi

Tue Jul 28 16:48:17 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.51.05    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   46C    P8    10W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive
 0E0B14F4-4F83-49EC-A0B7-BA77F8ECF64B.JPG
 0E9D8F19-942B-4C98-A6A9-06022E63432D.JPG
 1621555042.jpg
 1825E521-AA53-4FEA-A155-2C5B9DACBEBF.JPG
 1A07B3FE-03EC-4127-8369-3069142D9434.JPG
'1. Data and Signals.gdoc'
 2B0AF0F9-6C1F-4E95-8752-200D13AED246.JPG
 3611EF88-0805-4572-A393-235910452645.JPG
 51018219_2559457984095370_6715450653155524608_o.jpg
 5C731F5A-E5ED-45ED-B434-2E0B162BFD0C.JPG
 73F9D72B-DC77-4254-9A45-26A0929FC9D7.JPG
 7B1E92A8-5CE8-4697-9BAE-7E0964

**1) Clone the Darknet**



In [ ]:
!git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 8, done.
remote: Counting objects: 100% (8/8), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 14086 (delta 0), reused 1 (delta 0), pack-reused 14078
Receiving objects: 100% (14086/14086), 12.77 MiB | 15.48 MiB/s, done.
Resolving deltas: 100% (9580/9580), done.


**2) Compile Darknet using Nvidia GPU**


In [ ]:
# change makefile to have GPU and OPENCV enabled
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!make

/content/darknet
mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:926:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1127:13: warning: this ‘if’ clause does not guard... [-Wmisleading-indentation]
             if (iteration_old == 0)
           

**3) Configure Darknet network for training YOLO V3**

In [ ]:
!cp cfg/yolov3.cfg cfg/yolov3_training.cfg

In [ ]:
!sed -i 's/batch=1/batch=64/' cfg/yolov3_training.cfg
!sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov3_training.cfg
!sed -i 's/max_batches = 500200/max_batches = 4000/' cfg/yolov3_training.cfg
!sed -i '610 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '696 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '783 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '603 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '689 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '776 s@filters=255@filters=18@' cfg/yolov3_training.cfg

In [ ]:
# Create folder on google drive so that we can save there the weights
!mkdir "/mydrive/yolov3"

mkdir: cannot create directory ‘/mydrive/yolov3’: File exists


In [ ]:
!echo "Football" > data/obj.names
!echo -e 'classes= 1\ntrain  = data/train.txt\nvalid  = data/test.txt\nnames = data/obj.names\nbackup = /mydrive/yolov3' > data/obj.data
!mkdir data/obj

In [ ]:
# Download weights darknet model 53
!wget https://pjreddie.com/media/files/darknet53.conv.74

--2020-07-28 16:55:55--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74’

darknet53.conv.74   100%[===================>] 154.96M  1.21MB/s    in 1m 57s  

2020-07-28 16:57:52 (1.33 MB/s) - ‘darknet53.conv.74’ saved [162482580/162482580]



**4) Extract Images**

The images need to be inside a zip archive called "images.zip" and they need to be inside the folder "yolov3" on Google Drive

In [ ]:
!unzip /mydrive/yolov3/images.zip -d data/obj

Archive:  /mydrive/yolov3/images.zip
  inflating: data/obj/0b4c7316761f7ada.jpg  
  inflating: data/obj/0b4c7316761f7ada.txt  
  inflating: data/obj/0b666ad49ec29585.jpg  
  inflating: data/obj/0b666ad49ec29585.txt  
  inflating: data/obj/0bbc46eade0adc8e.jpg  
  inflating: data/obj/0bbc46eade0adc8e.txt  
  inflating: data/obj/0be4fb66e0cb1d13.jpg  
  inflating: data/obj/0be4fb66e0cb1d13.txt  
  inflating: data/obj/0bf1fb65d7bbc2cd.jpg  
  inflating: data/obj/0bf1fb65d7bbc2cd.txt  
  inflating: data/obj/0ca4595b20b42098.jpg  
  inflating: data/obj/0ca4595b20b42098.txt  
  inflating: data/obj/0e7a790a236013ac.jpg  
  inflating: data/obj/0e7a790a236013ac.txt  
  inflating: data/obj/0f40cf6a99e88f10.jpg  
  inflating: data/obj/0f40cf6a99e88f10.txt  
  inflating: data/obj/0f59253fd8155cfe.jpg  
  inflating: data/obj/0f59253fd8155cfe.txt  
  inflating: data/obj/01a4bf61960cfc43.jpg  
  inflating: data/obj/01a4bf61960cfc43.txt  
  inflating: data/obj/01b56b7284e31d8a.jpg  
  inflating: data/

In [ ]:
# We're going to convert the class index on the .txt files. As we're working with only one class, it's supposed to be class 0.
# If the index is different from 0 then we're going to change it.
import glob
import os
import re

txt_file_paths = glob.glob(r"data/obj/*.txt")
for i, file_path in enumerate(txt_file_paths):
    # get image size
    with open(file_path, "r") as f_o:
        lines = f_o.readlines()

        text_converted = []
        for line in lines:
            print(line)
            numbers = re.findall("[0-9.]+", line)
            print(numbers)
            if numbers:

                # Define coordinates
                text = "{} {} {} {} {}".format(0, numbers[1], numbers[2], numbers[3], numbers[4])
                text_converted.append(text)
                print(i, file_path)
                print(text)
        # Write file
        with open(file_path, 'w') as fp:
            for item in text_converted:
                fp.writelines("%s\n" % item)

0 0.517537375 0.0997499633967789 0.04407899999999998 0.06804699999999998

['0', '0.517537375', '0.0997499633967789', '0.04407899999999998', '0.06804699999999998']
0 data/obj/8f003a8eea47c3dc.txt
0 0.517537375 0.0997499633967789 0.04407899999999998 0.06804699999999998
0 0.7460933750000001 0.645680767203514 0.07519599999999993 0.1083459999999999

['0', '0.7460933750000001', '0.645680767203514', '0.07519599999999993', '0.1083459999999999']
1 data/obj/b6806bdf6874851d.txt
0 0.7460933750000001 0.645680767203514 0.07519599999999993 0.1083459999999999
0 0.543671875 0.42185860761347 0.05874999999999997 0.08255199999999992

['0', '0.543671875', '0.42185860761347', '0.05874999999999997', '0.08255199999999992']
2 data/obj/1fd00ac78bd1c443.txt
0 0.543671875 0.42185860761347 0.05874999999999997 0.08255199999999992
0 0.31581497435897443 0.870883 0.11051199999999994 0.12574900000000003

['0', '0.31581497435897443', '0.870883', '0.11051199999999994', '0.12574900000000003']
3 data/obj/1f605aa78f8f7bb2.

In [ ]:
import glob
images_list = glob.glob("data/obj/*.jpg")
print(images_list)

['data/obj/ff86e01a7fa0abaa.jpg', 'data/obj/cd9341fb7ef6981b.jpg', 'data/obj/04fd519e5a2fc553.jpg', 'data/obj/1f76484fd42cfc61.jpg', 'data/obj/8c9fad719a65bf6a.jpg', 'data/obj/b24a9c8c2e5df857.jpg', 'data/obj/91f59c521290376a.jpg', 'data/obj/16f43f22487ed11c.jpg', 'data/obj/1e13768826ca5ac1.jpg', 'data/obj/021190e5c1b3e146.jpg', 'data/obj/629e3fa49dbb483b.jpg', 'data/obj/ab783a1a39cd5e57.jpg', 'data/obj/cae92d77fca93812.jpg', 'data/obj/c760dd7ee7fb9795.jpg', 'data/obj/3af9637550e1f5a9.jpg', 'data/obj/94d2b7c1d2d09b6b.jpg', 'data/obj/28644a1f2bc7ab51.jpg', 'data/obj/f18e2a17bab0842c.jpg', 'data/obj/a47717ec5fd897b5.jpg', 'data/obj/e68ac4cdc5d5420c.jpg', 'data/obj/4665203255773d43.jpg', 'data/obj/77e94b5ec7b3ec50.jpg', 'data/obj/5337b8b56fc24576.jpg', 'data/obj/312bd5cac8d6bdcb.jpg', 'data/obj/40652a32a0447c8c.jpg', 'data/obj/34bf49849b852162.jpg', 'data/obj/036c4897f041a7a2.jpg', 'data/obj/16aed06e2ee7d3d8.jpg', 'data/obj/832718f3cd3cdd42.jpg', 'data/obj/32bfad3c36d49ae8.jpg', 'data/obj

In [ ]:
#Create training.txt file
file = open("data/train.txt", "w") 
file.write("\n".join(images_list)) 
file.close() 

**6) Start the training**

In [ ]:
#  training
!./darknet detector train data/obj.data cfg/yolov3_training.cfg darknet53.conv.74 -dont_show

Streaming output truncated to the last 5000 lines.
v3 (mse loss, Normalizer: (iou: 0.75, cls: 1.00) Region 82 Avg (IOU: 0.000000, GIOU: 0.000000), Class: 0.000000, Obj: 0.000000, No Obj: 0.000004, .5R: 0.000000, .75R: 0.000000, count: 1, class_loss = 0.000000, iou_loss = 0.000000, total_loss = 0.000000 
v3 (mse loss, Normalizer: (iou: 0.75, cls: 1.00) Region 94 Avg (IOU: 0.822597, GIOU: 0.822597), Class: 0.999992, Obj: 0.999494, No Obj: 0.000086, .5R: 1.000000, .75R: 1.000000, count: 1, class_loss = 0.000046, iou_loss = 0.020006, total_loss = 0.020052 
v3 (mse loss, Normalizer: (iou: 0.75, cls: 1.00) Region 106 Avg (IOU: 0.724503, GIOU: 0.708101), Class: 0.997589, Obj: 0.097588, No Obj: 0.000013, .5R: 1.000000, .75R: 0.000000, count: 1, class_loss = 0.247460, iou_loss = 0.063948, total_loss = 0.311408 
 total_bbox = 136017, rewritten_bbox = 0.138218 % 
v3 (mse loss, Normalizer: (iou: 0.75, cls: 1.00) Region 82 Avg (IOU: 0.000000, GIOU: 0.000000), Class: 0.000000, Obj: 0.000000, No Obj: